In [1]:
import torch 
import argparse
from utils import dotdict
from activation_dataset import setup_token_data
import wandb
import json
from datetime import datetime
from tqdm import tqdm
from einops import rearrange
import matplotlib.pyplot as plt
# from standard_metrics import run_with_model_intervention, perplexity_under_reconstruction, mean_nonzero_activations
# Create 
# # make an argument parser directly below
# parser = argparse.ArgumentParser()
# parser.add_argument("--model_name", type=str, default="EleutherAI/pythia-70m-deduped")
# parser.add_argument("--layer", type=int, default=4)
# parser.add_argument("--setting", type=str, default="residual")
# parser.add_argument("--l1_alpha", type=float, default=3e-3)
# parser.add_argument("--num_epochs", type=int, default=10)
# parser.add_argument("--model_batch_size", type=int, default=4)
# parser.add_argument("--lr", type=float, default=1e-3)
# parser.add_argument("--kl", type=bool, default=False)
# parser.add_argument("--reconstruction", type=bool, default=False)
# parser.add_argument("--dataset_name", type=str, default="NeelNanda/pile-10k")
# parser.add_argument("--device", type=str, default="cuda:4")

# args = parser.parse_args()
cfg = dotdict()
# cfg.model_name="EleutherAI/pythia-70m-deduped", "usvsnsp/pythia-6.9b-sft"
cfg.model_name="EleutherAI/pythia-70m-deduped"
cfg.target_name="EleutherAI/pythia-70m-deduped"
cfg.layers=[4]
cfg.setting="residual"
cfg.tensor_name="gpt_neox.layers.{layer}"
original_l1_alpha = 1e-3
cfg.l1_alpha=original_l1_alpha
cfg.sparsity=None
cfg.num_epochs=10
cfg.model_batch_size=4
cfg.lr=1e-3
cfg.kl=False
cfg.reconstruction=False
# cfg.dataset_name="NeelNanda/pile-10k"
cfg.dataset_name="Elriggs/openwebtext-100k"
cfg.device="cuda:0"
cfg.ratio = 4
cfg.seed = 0
# cfg.device="cpu"

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tensor_names = [cfg.tensor_name.format(layer=layer) for layer in cfg.layers]

In [3]:
# Load in the model
from transformers import AutoModelForCausalLM, AutoTokenizer
model = AutoModelForCausalLM.from_pretrained(cfg.model_name)
model = model.to(cfg.device)
tokenizer = AutoTokenizer.from_pretrained(cfg.model_name)

In [4]:
# Download the dataset
# TODO iteratively grab dataset?
cfg.max_length = 256
cfg.model_batch_size = 4
token_loader = setup_token_data(cfg, tokenizer, model, seed=cfg.seed)
num_tokens = cfg.max_length*cfg.model_batch_size*len(token_loader)
print(f"Number of tokens: {num_tokens}")

Found cached dataset parquet (/root/.cache/huggingface/datasets/Elriggs___parquet/Elriggs--openwebtext-100k-79076ecafee8a6d5/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
Loading cached processed dataset at /root/.cache/huggingface/datasets/Elriggs___parquet/Elriggs--openwebtext-100k-79076ecafee8a6d5/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-8401ec7d4dbd84d2_*_of_00008.arrow


Number of tokens: 112749568


In [5]:
# Run 1 datapoint on model to get the activation size
from baukit import Trace

text = "1"
tokens = tokenizer(text, return_tensors="pt").input_ids.to(cfg.device)
# Your activation name will be different. In the next cells, we will show you how to find it.
with torch.no_grad():
    with Trace(model, tensor_names[0]) as ret:
        _ = model(tokens)
        representation = ret.output
        # check if instance tuple
        if(isinstance(representation, tuple)):
            representation = representation[0]
        activation_size = representation.shape[-1]
print(f"Activation size: {activation_size}")

Activation size: 512


In [6]:
# Initialize New autoencoder
from autoencoders.learned_dict import TiedSAE, UntiedSAE, AnthropicSAE
from torch import nn
params = dict()
n_dict_components = activation_size*cfg.ratio
params["encoder"] = torch.empty((n_dict_components, activation_size), device=cfg.device)
nn.init.xavier_uniform_(params["encoder"])

params["decoder"] = torch.empty((n_dict_components, activation_size), device=cfg.device)
nn.init.xavier_uniform_(params["decoder"])

params["encoder_bias"] = torch.empty((n_dict_components,), device=cfg.device)
nn.init.zeros_(params["encoder_bias"])

params["shift_bias"] = torch.empty((activation_size,), device=cfg.device)
nn.init.zeros_(params["shift_bias"])

autoencoder = AnthropicSAE(  # TiedSAE, UntiedSAE, AnthropicSAE
    # n_feats = n_dict_components, 
    # activation_size=activation_size,
    encoder=params["encoder"],
    encoder_bias=params["encoder_bias"],
    decoder=params["decoder"],
    shift_bias=params["shift_bias"],
)
autoencoder.to_device(cfg.device)
autoencoder.set_grad()
# autoencoder.encoder.requires_grad = True
# autoencoder.encoder_bias.requires_grad = True
# autoencoder.decoder.requires_grad = True
# autoencoder.shift_bias.requires_grad = True
optimizer = torch.optim.Adam(
    [
        autoencoder.encoder, 
        autoencoder.encoder_bias,
        autoencoder.decoder,
        autoencoder.shift_bias,
    ], lr=cfg.lr)

In [7]:
# Set target sparsity to 10% of activation_size if not set
if cfg.sparsity is None:
    cfg.sparsity = int(activation_size*0.1)
    print(f"Target sparsity: {cfg.sparsity}")

target_lower_sparsity = cfg.sparsity - 5.0
target_upper_sparsity = cfg.sparsity + 5.0
adjustment_factor = 0.1  # You can set this to whatever you like

Target sparsity: 51


In [8]:
original_bias = autoencoder.encoder_bias.clone().detach()
# Wandb setup
secrets = json.load(open("secrets.json"))
wandb.login(key=secrets["wandb_key"])
start_time = datetime.now().strftime("%Y%m%d-%H%M%S")
wandb_run_name = f"{cfg.model_name}_{start_time[4:]}_{cfg.sparsity}"  # trim year
print(f"wandb_run_name: {wandb_run_name}")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


wandb: Currently logged in as: benw8888. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


wandb_run_name: EleutherAI/pythia-70m-deduped_1011-032433_51


In [14]:
wandb.init(project="sparse coding", config=dict(cfg), name=wandb_run_name)

In [13]:
# import os
# class ActivationSaver():    
#     def __init__(self, batches_per_file=5_000, folder_path="saved_activations"):
#         self.total_batches = 0
#         self.batches_per_file = batches_per_file
#         self.folder_path = folder_path
        
#         self.files = []
#         self.batch_buffer = []
#         if not os.path.exists(folder_path):
#             os.makedirs(folder_path)
            
#     def save_batch(self, batch):
#         if len(self.batch_buffer) >= self.batches_per_file:
#             new_file = f"{self.folder_path}/activations_{len(self.files)+1}.pkl"
#             torch.save(self.batch_buffer, new_file)
#             self.batch_buffer = []
#             self.files.append(new_file)
            
#         self.batch_buffer.append(batch)
#         self.total_batches +=1
    
#     def save_buffer(self):
#         new_file = f"{self.folder_path}/activations_{len(self.files)+1}.pkl"
#         torch.save(self.batch_buffer, new_file)
#         self.batch_buffer = []
    
#     def __len__(self):
#         return self.total_batches
        
#     def load_file(self, i):
#         return torch.load(self.files[i])
    
#     def generator(self):
#         cur_file = 0
#         while cur_file < len(self.files):
#             for batch in self.load_file(cur_file):
#                 yield batch
#             cur_file += 1
#         for batch in self.batch_buffer:
#             yield batch
# activation_saver = ActivationSaver(batches_per_file=5_000, folder_path="saved_activations/base")

In [15]:
dead_features = torch.zeros(autoencoder.encoder.shape[0])
total_activations = torch.zeros(autoencoder.encoder.shape[0])
max_num_tokens = 100_000_000
# Freeze model parameters 
model.eval()
model.requires_grad_(False)
model.to(cfg.device)
last_encoder = autoencoder.encoder.clone().detach()
for i, batch in enumerate(tqdm(token_loader)):
    tokens = batch["input_ids"].to(cfg.device)
    with torch.no_grad(): # As long as not doing KL divergence, don't need gradients for model
        with Trace(model, tensor_names[0]) as ret:
            _ = model(tokens)
            representation = ret.output
            if(isinstance(representation, tuple)):
                representation = representation[0]
    layer_activations = rearrange(representation, "b seq d_model -> (b seq) d_model")
    # activation_saver.save_batch(layer_activations.clone().cpu().detach())

    c = autoencoder.encode(layer_activations)
    x_hat = autoencoder.decode(c)
    
    reconstruction_loss = (x_hat - layer_activations).pow(2).mean()
    l1_loss = torch.norm(c, 1, dim=-1).mean()
    total_loss = reconstruction_loss + cfg.l1_alpha*l1_loss

    dead_features += c.sum(dim=0).cpu()
    total_activations += c.sum(dim=0).cpu()
    if (i % 500 == 0): # Check here so first check is model w/o change
        # self_similarity = torch.cosine_similarity(c, last_encoder, dim=-1).mean().cpu().item()
        # Above is wrong, should be similarity between encoder and last encoder
        self_similarity = torch.cosine_similarity(autoencoder.encoder, last_encoder, dim=-1).mean().cpu().item()
        last_encoder = autoencoder.encoder.clone().detach()
        num_tokens_so_far = i*cfg.max_length*cfg.model_batch_size
        with torch.no_grad():
            sparsity = (c != 0).float().mean(dim=0).sum().cpu().item()
            # Count number of dead_features are zero
            num_dead_features = (dead_features == 0).sum().item()
        print(f"Sparsity: {sparsity:.1f} | Dead Features: {num_dead_features} | Total Loss: {total_loss:.2f} | Reconstruction Loss: {reconstruction_loss:.2f} | L1 Loss: {cfg.l1_alpha*l1_loss:.2f} | l1_alpha: {cfg.l1_alpha:.2e} | Tokens: {num_tokens_so_far} | Self Similarity: {self_similarity:.2f}")
        wandb.log({
            'Sparsity': sparsity,
            'Dead Features': num_dead_features,
            'Total Loss': total_loss.item(),
            'Reconstruction Loss': reconstruction_loss.item(),
            'L1 Loss': (cfg.l1_alpha*l1_loss).item(),
            'l1_alpha': cfg.l1_alpha,
            'Tokens': num_tokens_so_far,
            'Self Similarity': self_similarity
        })
        
        dead_features = torch.zeros(autoencoder.encoder.shape[0])
        
        if(num_tokens_so_far > max_num_tokens):
            print(f"Reached max number of tokens: {max_num_tokens}")
            break
    
    resample_period = 500
    if (i % resample_period == 0):
        # RESAMPLING
        with torch.no_grad():
            # Count number of dead_features are zero
            num_dead_features = (total_activations == 0).sum().item()
            
        if num_dead_features > 0:
            # hyperparams:
            max_resample_tokens = 1000 # the number of token activations that we consider for inserting into the dictionary
            # compute loss of model on random subset of inputs
            resample_loader = setup_token_data(cfg, tokenizer, model, seed=i)
            num_resample_data = 0

            resample_activations = torch.empty(0, activation_size)
            resample_losses = torch.empty(0)

            for resample_batch in resample_loader:
                resample_tokens = resample_batch["input_ids"].to(cfg.device)
                with torch.no_grad(): # As long as not doing KL divergence, don't need gradients for model
                    with Trace(model, tensor_names[0]) as ret:
                        _ = model(resample_tokens)
                        representation = ret.output
                        if(isinstance(representation, tuple)):
                            representation = representation[0]
                layer_activations = rearrange(representation, "b seq d_model -> (b seq) d_model")
                resample_activations = torch.cat((resample_activations, layer_activations.detach().cpu()), dim=0)

                c = autoencoder.encode(layer_activations)
                x_hat = autoencoder.decode(c)
                
                reconstruction_loss = (x_hat - layer_activations).pow(2).mean(dim=-1)
                l1_loss = torch.norm(c, 1, dim=-1)
                temp_loss = reconstruction_loss + cfg.l1_alpha*l1_loss
                
                resample_losses = torch.cat((resample_losses, temp_loss.detach().cpu()), dim=0)
                
                num_resample_data +=layer_activations.shape[0]
                if num_resample_data > max_resample_tokens:
                    break

                
            # sample num_dead_features vectors of input activations
            probabilities = resample_losses**2
            sampled_indices = torch.multinomial(probabilities, num_dead_features)
            new_vectors = resample_activations[sampled_indices]

            # calculate average encoder norm of alive neurons
            alive_neurons = list((total_activations!=0))
            modified_columns = total_activations==0
            avg_norm = autoencoder.encoder.data[alive_neurons].norm(dim=-1).mean()

            # replace dictionary and encoder weights with vectors
            new_vectors = new_vectors / new_vectors.norm(dim=1, keepdim=True)
            
            params_to_modify = [autoencoder.encoder, autoencoder.encoder_bias]

            current_weights = autoencoder.encoder.data
            current_weights[modified_columns] = (new_vectors.to(cfg.device) * avg_norm * 0.2)
            autoencoder.encoder.data = current_weights

            current_weights = autoencoder.encoder_bias.data
            current_weights[modified_columns] = 0
            autoencoder.encoder_bias.data = current_weights
            
            if hasattr(autoencoder, 'decoder'):
                current_weights = autoencoder.decoder.data
                current_weights[modified_columns] = new_vectors.to(cfg.device)
                autoencoder.decoder.data = current_weights
                params_to_modify += [autoencoder.decoder]

            for param_group in optimizer.param_groups:
                for param in param_group['params']:
                    if any(param is d_ for d_ in params_to_modify):
                        # Extract the corresponding rows from m and v
                        m = optimizer.state[param]['exp_avg']
                        v = optimizer.state[param]['exp_avg_sq']
                        
                        # Update the m and v values for the modified columns
                        m[modified_columns] = 0  # Reset moving average for modified columns
                        v[modified_columns] = 0  # Reset squared moving average for modified columns
        
        total_activations = torch.zeros(autoencoder.encoder.shape[0])

    optimizer.zero_grad()
    total_loss.backward()
    optimizer.step()

    # # Running sparsity check
    # if(num_tokens_so_far > 500000):
    #     if(i % 200 == 0):
    #         with torch.no_grad():
    #             sparsity = (c != 0).float().mean(dim=0).sum().cpu().item()
    #         if sparsity > target_upper_sparsity:
    #             cfg.l1_alpha *= (1 + adjustment_factor)
    #         elif sparsity < target_lower_sparsity:
    #             cfg.l1_alpha *= (1 - adjustment_factor)
    #         # print(f"Sparsity: {sparsity:.1f} | l1_alpha: {cfg.l1_alpha:.2e}")

  0%|          | 3/110107 [00:00<1:02:59, 29.13it/s]

  0%|          | 8/110107 [00:00<47:14, 38.84it/s]  

Sparsity: 1016.3 | Dead Features: 0 | Total Loss: 1.56 | Reconstruction Loss: 1.19 | L1 Loss: 0.37 | l1_alpha: 1.00e-03 | Tokens: 0 | Self Similarity: 1.00


  0%|          | 508/110107 [00:11<40:26, 45.16it/s]

Sparsity: 145.7 | Dead Features: 0 | Total Loss: 0.14 | Reconstruction Loss: 0.09 | L1 Loss: 0.05 | l1_alpha: 1.00e-03 | Tokens: 512000 | Self Similarity: 0.79


...

Sparsity: 133.1 | Dead Features: 0 | Total Loss: 0.10 | Reconstruction Loss: 0.06 | L1 Loss: 0.04 | l1_alpha: 1.00e-03 | Tokens: 50176000 | Self Similarity: 0.97


 45%|████▍     | 49506/110107 [20:20<22:00, 45.88it/s]

Sparsity: 140.0 | Dead Features: 0 | Total Loss: 0.09 | Reconstruction Loss: 0.05 | L1 Loss: 0.04 | l1_alpha: 1.00e-03 | Tokens: 50688000 | Self Similarity: 0.97


 45%|████▌     | 50000/110107 [20:37<24:47, 40.42it/s]


RuntimeError: [enforce fail at inline_container.cc:337] . unexpected pos 1151820160 vs 1151820048

In [17]:
model_save_name = cfg.model_name.split("/")[-1]
save_name = f"{model_save_name}_sp{cfg.sparsity}_r{cfg.ratio}_{tensor_names[0]}"  # trim year

# Make directory traiend_models if it doesn't exist
import os
if not os.path.exists("trained_models"):
    os.makedirs("trained_models")
# Save model
torch.save(autoencoder, f"trained_models/{save_name}.pt")

# if not os.path.exists("activations"):
#     os.makedirs("activations")
# # Save model
# torch.save(saved_activations[:-1], f"activations/{save_name}.pt")

In [18]:
wandb.finish()

Dead Features,▁▁▁▁▁▂▃▂█▅▃▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
L1 Loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Reconstruction Loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Self Similarity,█▁▃▃▃▃▂▄▂▄▅▄▄▅▅▅▅▄▂▅▅▅▅▄▄▅▄▅▅▅▄▅▅▅▅▄▅▅▄▅
Sparsity,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Tokens,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
Total Loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
l1_alpha,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Dead Features,0
L1 Loss,0.04268
Reconstruction Loss,0.04672


In [69]:
target_model = AutoModelForCausalLM.from_pretrained(cfg.target_name).cpu()

model_save_name = cfg.model_name.split("/")[-1]
save_name = f"{model_save_name}_sp{cfg.sparsity}_r{cfg.ratio}_{tensor_names[0]}"  # trim year
autoencoder = torch.load(f"trained_models/{save_name}.pt")

In [70]:
# Initialize New autoencoder
from autoencoders.learned_dict import TiedSAE, UntiedSAE, AnthropicSAE, TransferSAE
from torch import nn

# params["decoder"] = torch.empty((n_dict_components, activation_size), device=cfg.device)
# nn.init.xavier_uniform_(params["decoder"])

params["decoder_bias"] = torch.empty((activation_size,), device=cfg.device)
nn.init.zeros_(params["decoder_bias"])

transfer_autoencoder = TransferSAE(
    # n_feats = n_dict_components, 
    # activation_size=activation_size,
    autoencoder,
    decoder=autoencoder.get_learned_dict().detach().clone(),
    decoder_bias=autoencoder.shift_bias.detach().clone(),
)
transfer_autoencoder.to_device(cfg.device)

# Set gradient to true for decoder only- only training decoder on transfer
transfer_autoencoder.set_grad()
optimizer = torch.optim.Adam(
    [
        transfer_autoencoder.decoder,
        transfer_autoencoder.decoder_bias,
    ], lr=cfg.lr)


In [71]:
# Wandb setup
secrets = json.load(open("secrets.json"))
wandb.login(key=secrets["wandb_key"])
start_time = datetime.now().strftime("%Y%m%d-%H%M%S")
wandb_run_name = f"{cfg.target_name}_transfer_{start_time[4:]}_{cfg.sparsity}"  # trim year
print(f"wandb_run_name: {wandb_run_name}")
wandb.init(project="sparse coding", config=dict(cfg), name=wandb_run_name)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


wandb_run_name: EleutherAI/pythia-70m-deduped_transfer_1011-042101_51


Dead Features,█▁▁▁▁
Reconstruction Loss,█▁▁▁▁
Self Similarity,█▁▇▇▇
Sparsity,▆█▆▂▁
Tokens,▁▃▅▆█
Dead Features,0
Reconstruction Loss,0.04103
Self Similarity,0.98778
Sparsity,138.9082
Tokens,2048000


In [72]:
def get_activations(model, inputs):
    acts = []
    for tokens in inputs:
        with torch.no_grad(): # As long as not doing KL divergence, don't need gradients for model
            with Trace(model, tensor_names[0]) as ret:
                _ = model(tokens)
                representation = ret.output
                if(isinstance(representation, tuple)):
                    representation = representation[0]
        layer_activations = rearrange(representation, "b seq d_model -> (b seq) d_model")
        acts.append(layer_activations.cpu())
    return acts

In [73]:
# Training transfer autoencoder
token_loader = setup_token_data(cfg, tokenizer, model, seed=cfg.seed)
dead_features = torch.zeros(transfer_autoencoder.encoder.shape[0])
max_num_tokens = 100_000_000
# Freeze model parameters 
model = model.to(cfg.device)
target_model = target_model.cpu()
target_model.eval()
target_model.requires_grad_(False)

last_decoder = transfer_autoencoder.decoder.clone().detach()
model_on_gpu = True

saved_inputs = []
i = 0 # counts all optimization steps
for k, (batch) in enumerate(token_loader):
    saved_inputs.append(batch["input_ids"].to(cfg.device))
    
    if (k+1)%500==0:
        # compute base and target model activations
        if model_on_gpu:
            base_activations = get_activations(model, saved_inputs)
            model = model.cpu()
            target_model = target_model.to(cfg.device)
        target_activations = get_activations(target_model, saved_inputs)
        if not model_on_gpu:
            target_model = target_model.cpu()
            model = model.to(cfg.device)
            base_activations = get_activations(model, saved_inputs)
        model_on_gpu = not model_on_gpu

        # wipe saved inputs
        saved_inputs = []
        
        # train autoencoder on activations:
        for (base_activation, target_activation) in (zip(base_activations, target_activations)):
            c = transfer_autoencoder.encode(base_activation.to(cfg.device))
            x_hat = transfer_autoencoder.decode(c)
            
            reconstruction_loss = (x_hat - target_activation.to(cfg.device)).pow(2).mean()
            total_loss = reconstruction_loss # NO L1 LOSS

            dead_features += c.sum(dim=0).cpu()
            if (i % 500 == 0): # Check here so first check is model w/o change
                self_similarity = torch.cosine_similarity(transfer_autoencoder.decoder, last_decoder, dim=-1).mean().cpu().item()
                last_decoder = transfer_autoencoder.decoder.clone().detach()
                num_tokens_so_far = i*cfg.max_length*cfg.model_batch_size
                with torch.no_grad():
                    sparsity = (c != 0).float().mean(dim=0).sum().cpu().item()
                    # Count number of dead_features are zero
                    num_dead_features = (dead_features == 0).sum().item()
                print(f"Sparsity: {sparsity:.1f} | Dead Features: {num_dead_features} | Reconstruction Loss: {reconstruction_loss:.2f} | Tokens: {num_tokens_so_far} | Self Similarity: {self_similarity:.2f}")
                wandb.log({
                    'Sparsity': sparsity,
                    'Dead Features': num_dead_features,
                    'Reconstruction Loss': reconstruction_loss.item(),
                    'Tokens': num_tokens_so_far,
                    'Self Similarity': self_similarity
                })
                dead_features = torch.zeros(transfer_autoencoder.encoder.shape[0])
                
                if(num_tokens_so_far > max_num_tokens):
                    print(f"Reached max number of tokens: {max_num_tokens}")
                    break

            optimizer.zero_grad()
            total_loss.backward()
            optimizer.step()
            i+=1

            # # Running sparsity check
            # if(num_tokens_so_far > 500000):
            #     if(i % 200 == 0):
            #         with torch.no_grad():
            #             sparsity = (c != 0).float().mean(dim=0).sum().cpu().item()
            #         if sparsity > target_upper_sparsity:
            #             cfg.l1_alpha *= (1 + adjustment_factor)
            #         elif sparsity < target_lower_sparsity:
            #             cfg.l1_alpha *= (1 - adjustment_factor)
            #         # print(f"Sparsity: {sparsity:.1f} | l1_alpha: {cfg.l1_alpha:.2e}")

Found cached dataset parquet (/root/.cache/huggingface/datasets/Elriggs___parquet/Elriggs--openwebtext-100k-79076ecafee8a6d5/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
Loading cached processed dataset at /root/.cache/huggingface/datasets/Elriggs___parquet/Elriggs--openwebtext-100k-79076ecafee8a6d5/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-8401ec7d4dbd84d2_*_of_00008.arrow


Sparsity: 143.0 | Dead Features: 1 | Reconstruction Loss: 0.05 | Tokens: 0 | Self Similarity: 1.00
Sparsity: 145.0 | Dead Features: 0 | Reconstruction Loss: 0.04 | Tokens: 512000 | Self Similarity: 0.98
Sparsity: 143.0 | Dead Features: 0 | Reconstruction Loss: 0.04 | Tokens: 1024000 | Self Similarity: 0.99
Sparsity: 139.8 | Dead Features: 0 | Reconstruction Loss: 0.04 | Tokens: 1536000 | Self Similarity: 0.99
Sparsity: 138.9 | Dead Features: 0 | Reconstruction Loss: 0.04 | Tokens: 2048000 | Self Similarity: 0.99
Sparsity: 140.2 | Dead Features: 0 | Reconstruction Loss: 0.04 | Tokens: 2560000 | Self Similarity: 0.99
Sparsity: 136.2 | Dead Features: 0 | Reconstruction Loss: 0.04 | Tokens: 3072000 | Self Similarity: 0.99


KeyboardInterrupt: 

In [ ]:
model_save_name = cfg.target_name.split("/")[-1]
save_name = f"{model_save_name}_transfer_sp{cfg.sparsity}_r{cfg.ratio}_{tensor_names[0]}"  # trim year

# Make directory traiend_models if it doesn't exist
import os
if not os.path.exists("trained_models"):
    os.makedirs("trained_models")
# Save model
torch.save(transfer_autoencoder, f"trained_models/{save_name}.pt")

In [74]:
wandb.finish()

Dead Features,█▁▁▁▁▁▁
Reconstruction Loss,█▆▅▅▄▃▁
Self Similarity,█▁▃▃▃▃▄
Sparsity,▆█▆▄▃▄▁
Tokens,▁▂▃▅▆▇█
Dead Features,0
Reconstruction Loss,0.03738
Self Similarity,0.98854
Sparsity,136.16797
Tokens,3072000


In [75]:
# Autoencoder vs Transfer Autoencoder Similarity
torch.cosine_similarity(autoencoder.decoder,transfer_autoencoder.decoder, dim=0)

tensor([0.9663, 0.9664, 0.9666, 0.9672, 0.9662, 0.9652, 0.9650, 0.9667, 0.9665,
        0.9600, 0.9669, 0.9665, 0.9662, 0.9661, 0.9655, 0.9668, 0.9653, 0.9691,
        0.9685, 0.9654, 0.9663, 0.9629, 0.9655, 0.9678, 0.9656, 0.9702, 0.9649,
        0.9666, 0.9677, 0.9669, 0.9678, 0.9674, 0.9665, 0.9662, 0.9642, 0.9675,
        0.9689, 0.9663, 0.9660, 0.9657, 0.9652, 0.9638, 0.9687, 0.9644, 0.9650,
        0.9674, 0.9662, 0.9680, 0.9657, 0.9666, 0.9668, 0.9667, 0.9663, 0.9690,
        0.9658, 0.9665, 0.9675, 0.9632, 0.9659, 0.9678, 0.9669, 0.9663, 0.9660,
        0.9683, 0.9685, 0.9681, 0.9670, 0.9652, 0.9647, 0.9668, 0.9661, 0.9678,
        0.9678, 0.9675, 0.9663, 0.9661, 0.9684, 0.9669, 0.9682, 0.9668, 0.9641,
        0.9681, 0.9646, 0.9665, 0.9678, 0.9668, 0.9660, 0.9663, 0.9658, 0.9665,
        0.9649, 0.9663, 0.9665, 0.9665, 0.9650, 0.9674, 0.9630, 0.9635, 0.9669,
        0.9681, 0.9676, 0.9646, 0.9648, 0.9659, 0.9671, 0.9658, 0.9668, 0.9668,
        0.9643, 0.9669, 0.9664, 0.9636, 